# CGRA 1st Assignment

Diogo Seabra Mota Henriques de Gouveia - 2018279473

Manuel Alberto Dionísio dos Santos - 2019231352

In [ ]:
%%html               
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecshader.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecapp.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/cgraobject.js"></script>
<script src='https://git.io/glm-js.min.js'></script>

In [ ]:
%%html
<script id="my-vertex-shader" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
uniform mat4 MVP;

varying  vec3 ex_Color;

void main(void)
{  
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);

    ex_Color = in_Color;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shader" type="x-shader/x-fragment">
precision mediump float;

varying  vec3 ex_Color;

void main(void)
{  
    gl_FragColor = vec4(ex_Color,1.0);   
}
</script>

# ================== A.1 ==================

# Cube
To create an unitary cube (centered in the origin and of size 1), we identified the 8 vertices of it in space. Ex: [x = 0,5 ; y = 0,5 ; z = 0,5].

After that, we created a lookup table of all the 36 vertices necessary to create a cube solely composed of triangles. By traversing it, we populated the vertex buffer.

To populate the colors buffer, we did a similar aproach, but instead of the lookup table, we just pushed 2 intercalated vectors of colors into the buffer.

We also created a method to dinamically change the cube colors.

Finally, we instantiated a cube object and appiled transformations to it.

In [ ]:
%%html

<canvas id="CubeCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="CubeApp">

class cube extends CGRAobject
{
    // ===== Constructor
    constructor(glcontext, col1 = [1.0, 0.0, 0.0], col2 = [0.0, 0.0, 1.0])
    {
        // ===== Initialize the parent class
        super(glcontext); 
        
        // ===== Define COnstants
        this.numvertices = 36;
        this.size = 1;
        let dim = this.size/2;
        
        // ===== List vertices
        var verticesCube = [[ -dim, -dim,  dim],
                             [-dim,  dim,  dim],
                             [ dim,  dim,  dim],
                             [ dim, -dim,  dim],
                             [-dim, -dim, -dim],
                             [-dim,  dim, -dim],
                             [ dim,  dim, -dim],
                             [ dim, -dim, -dim]]; 

        // ===== Lookup table
        var vertices = [];
        var indexes = [
            0, 1, 2,  2, 3, 0,  // Front face
            6, 7, 3,  2, 3, 6,  // Right face
            6, 7, 4,  4, 5, 6,  // Back  face
            1, 0, 4,  4, 5, 1,  // Left  face
            1, 6, 2,  3, 4, 0,  // Up    face
            1, 6, 5,  3, 4, 7]; // Down  face
        
        // ===== Create structure to populate buffer
        for (var i = 0; i < indexes.length; i++)
        {
            vertices = vertices.concat(verticesCube[indexes[i]]);
        } 

        // ===== Assigning colors
        var colors = [];
        this.color1 = [col1, col1, col2].flat();
        this.color2 = [col2, col2, col1].flat();
        for (var i=0; i < (this.numvertices/3)/2; i++)
        {
            colors.push(this.color1);
            colors.push(this.color2);
        }
        
        
        // ===== Creating and configuring buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
    
    // ===== Function to dinamically change cube colors
    setColors(newColor1, newColor2)
    {
        this.color1 = [newColor1, newColor1, newColor2].flat();
        this.color2 = [newColor2, newColor2, newColor1].flat();
        let colors = [];
        for (var i=0; i < (this.numvertices/3)/2; i++)
        {
            colors.push(this.color1);
            colors.push(this.color2);
        }
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
}


class CubeApp extends DEECapp
{
    counter = 0;
    initialize()
    {
        // ===== Perform initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9, 0.9, 0.9, 1.0);
        
        // ===== Instantiate a cube
        this.myCube = new cube(this.gl, [1, 0, 0], [0, 0, 1]);
        this.myCube.setShader(this.shaderprog);

        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,100);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(3, 3, 5): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(3,3,5),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;

        this.shaderprog.startUsing();
                
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(1.0, 1.0, 1.0);
        model = glm.scale(model, scale);
        // ===== Translate
        var translation = glm.vec3(0,Math.sin(this.counter/5),0);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter*5);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.myCube.setModelTransformation(model);      
        
        // ===== Draw Cube
        this.myCube.drawit(this.viewM,this.projectionM);
        
        // ===== Play with colors
        var speed = this.counter/10;
        var col1 = Math.abs(Math.sin(speed%(2*Math.PI)));
        var col2 = Math.abs(Math.cos(speed%(2*Math.PI)));
        this.myCube.setColors([1.0*col1, 0.0*col1, 0.0*col1],
                              [0.0*col2, 0.0*col2, 1.0*col2]);
    }
}

var app = new CubeApp('CubeCanvas');

app.run();
</script>

# Sphere

To create a sphere of diameter 1, we created N slices of it, composed of N quadrilateral's each (2 triangles for each quadrilateral, except on top and bottom ends, that have only 1 each).

We pushd each slice into the vertex buffer and at the same time the corresponding colors to the color buffer

Finally, we instantiated a sphere object and moved the camera around it.

In [ ]:
%%html

<canvas id="SphereCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="SphereApp">

class sphere extends CGRAobject
{
    center = glm.vec4(0, 0, 0, 1); 
    north_pole = glm.vec4(0, 0.5, 0, 1); 
    
    // ===== Constructor
    constructor(glcontext, col1 = [1.0, 1.0, 1.0], col2 = [0.0, 0.0, 0.0], faces = 10)
    {
        // Initialize the parent class
        super(glcontext); 
        
        // ===== Constants
        // Number of faces (3 to inf)
        this.sides = faces;
        // ===== Number of vertices
        this.numvertices = (6*this.sides + (6*this.sides)*(this.sides-2));
        this.length = 1; this.radius = 0.5;
        
        // ===== Create circunferences (360/N sides)
        let slices = [];   
        for (let i = 0; i < this.sides; i++)
        { 
            let theta = i*2*Math.PI/this.sides, aux = [];
            for (let j = 0; j < this.sides-1; j++)
            {     
                aux.push(
                    [(Math.sin((j+1)*Math.PI/this.sides)*this.radius)*Math.cos(theta),
                     Math.cos((j+1)*Math.PI/this.sides)*this.radius,
                    (Math.sin((j+1)*Math.PI/this.sides)*this.radius)*Math.sin(theta)]);
            }   
            slices.push(aux);
        }
        slices.push([[0,this.radius,0],[0,-1*this.radius,0]]);
        
        
        var vertices = [];
        var colors = [];
        
        // ===== Calculate gradient of colors
        var dR = 255*((col2[0] - col1[0]) / this.sides);
        var dG = 255*((col2[1] - col1[1]) / this.sides);
        var dB = 255*((col2[2] - col1[2]) / this.sides);
        var gradient = [];
        for (let i = 0; i < this.sides + 1; i++)
        {
            var Ri = Math.round(col1[0]*255 + i * dR)/255;
            var Gi = Math.round(col1[1]*255 + i * dG)/255;
            var Bi = Math.round(col1[2]*255 + i * dB)/255;
            gradient.push([Ri, Gi, Bi]);
        }
        gradient[0] = col1;
        gradient[this.sides] = col2;
        
        // ===== Create vertex and color data to populate buffers
        for (let i = 0; i < this.sides; i++)
        {            
            // Top side
            vertices.push(slices[this.sides][0]);
            vertices.push(slices[i][0]);
            vertices.push(slices[(i+1)%this.sides][0]);
            colors.push([gradient[0],gradient[1],gradient[1]].flat());
            
            // Middle sections
            for (let j = 0; j < this.sides - 2; j++)
            {
                vertices.push(slices[i][j]);
                vertices.push(slices[(i+1)%this.sides][j]);
                vertices.push(slices[i][j+1]);
                colors.push([gradient[j+1],gradient[j+1],gradient[j+2]].flat());

                vertices.push(slices[i][j+1]);
                vertices.push(slices[(i+1)%this.sides][j]);
                vertices.push(slices[(i+1)%this.sides][j+1]);
                colors.push([gradient[j+2],gradient[j+1],gradient[j+2]].flat());
            }
            
            // Down side
            vertices.push(slices[i][this.sides - 2]);
            vertices.push(slices[(i+1)%this.sides][this.sides - 2]);
            vertices.push(slices[this.sides][1]);
            colors.push([gradient[this.sides-1],gradient[this.sides-1],gradient[this.sides]].flat());
        }
        
        // ===== Populate Buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices.flat()), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
    
    // ===== Funcion to register center and north of sphere (WIP)
    update_pos(parentMat4 = glm.mat4(1.0))
    {
        var globalMat = parentMat4['*'](this.modelMat);
        
        //console.log(globalMat);
        
        this.center = globalMat['*'](this.center);
        //console.log("Center: " + this.center);
        this.north_pole = globalMat['*'](this.north_pole);
        //console.log("North: " + this.center);
    }
}


class SphereApp extends DEECapp
{
    counter=0;
    initialize()
    {
        // ===== Perform initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9,0.9,0.9,1.0);
        
        // ===== Create sphere
        this.my_Sphere = new sphere(this.gl,[0.95, 0.95, 0] ,[0.8, 0.8, 0.9], 20);
        this.my_Sphere.setShader(this.shaderprog);

        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,10);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        //this.viewM = glm.lookAt(glm.vec3(2,2,0),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        this.shaderprog.startUsing();
        
        // ===== Draw sphere
        this.my_Sphere.drawit(this.viewM,this.projectionM);
        
        // ===== Change view matrix
        this.viewM = glm.lookAt(glm.vec3(2,2*Math.sin(this.counter/50),0),glm.vec3(0,0,0),glm.vec3(0,1,0));
    }
}

var app = new SphereApp('SphereCanvas');

app.run();
</script>

# Cylinder / Prism

To create a cylinder of base 1, we created 2 bases (up and down) and conected each "face" of them with 2 triangles. In terms of division of the bases, the procedure is similar to the sphere. In this way, is possible to create solids from 3 to infinite faces. That means we can create a prism (3 faces), a "paralelepípedo"/cube (4 faces) or a clinder (5 to inf faces).

We also created a method to dinamically change the cylinder colors.

Finally, we instantiated a cylinder object and rotated it around the Z axis.

In [ ]:
%%html

<canvas id="CylinderCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="CylinderApp">

class cylinder extends CGRAobject
{
    // ===== Constructor
    constructor(glcontext, col1 = [0.0, 1.0, 0.0], col2 = [0.0, 0.0, 1.0], faces = 10)
    {
        // Initialize the parent class
        super(glcontext); 
        
        // ===== Constants
        // Number of faces (3 to inf)
        this.sides = faces;
        this.numvertices = this.sides*6 + this.sides*3*2;
        this.length = 1; this.radius = 0.5;
        
        // ===== Calculate top and bottom bases coordinates
        let up_side = [], down_side = [];   
        for (let i = 0; i < this.sides; i++)
        { 
            let theta = i*2*Math.PI/this.sides;
            up_side.push([this.radius*Math.cos(theta),
                          (this.length)/2,
                          this.radius*Math.sin(theta)]);
            
            down_side.push([this.radius*Math.cos(theta),
                            -1*(this.length)/2,
                            this.radius*Math.sin(theta)]);
        }
        
        // ===== Calculate vertices
        var vertices = [];
        for (let i = 0; i < this.sides; i++)
        {   
            vertices.push(up_side[i]);
            vertices.push(up_side[(i+1)%this.sides]);
            vertices.push(down_side[i]);
            
            vertices.push(down_side[i]);
            vertices.push(down_side[(i+1)%this.sides]);
            vertices.push(up_side[(i+1)%this.sides]);
            
            vertices.push(up_side[i]);
            vertices.push(up_side[(i+1)%this.sides]);
            vertices.push([0,(this.length)/2,0]);
            
            vertices.push(down_side[i]);
            vertices.push(down_side[(i+1)%this.sides]);
            vertices.push([0,-1*(this.length)/2,0]);
        }

        // ===== Assign colors
        var colors = [];
        this.color1 = [col1, col1, col2].flat();
        this.color2 = [col2, col2, col1].flat();
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.color1);
            colors.push(this.color2);
        }
        /*var alpha = 0;
        for (let i=0; i < this.numvertices; i++)
        {
            colors.push(col1[0]*alpha + (1*alpha)*col2[0]);
            colors.push(col1[1]*alpha + (1*alpha)*col2[1]);
            colors.push(col1[2]*alpha + (1*alpha)*col2[2]);
            alpha += 1.0/this.numvertices;
        }*/
        
        // ===== Populate vertex and color buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices.flat()), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
    
    // ===== Function to dynamically change the cylinder colors
    setColors(newColor1, newColor2)
    {
        this.color1 = [newColor1, newColor1, newColor2].flat();
        this.color2 = [newColor2, newColor2, newColor1].flat();
        let colors = [];
        for (var i=0; i < (this.numvertices/3)/2; i++)
        {
            colors.push(this.color1);
            colors.push(this.color2);
        }
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
}


class CylinderApp extends DEECapp
{
    counter=0;
    initialize()
    {
        // ===== Perform initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9, 0.9, 0.9, 1.0);
        
        // ===== Create cylinder
        this.my_Cylinder = new cylinder(this.gl);
        this.my_Cylinder.setShader(this.shaderprog);

        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,10);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(2,1,0),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        this.shaderprog.startUsing();
        
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(1.0);
        model = glm.scale(model, scale);
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(0.0, 0.0, 1.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.my_Cylinder.setModelTransformation(model); 
        
        // ==== Draw
        this.my_Cylinder.drawit(this.viewM,this.projectionM);
        
        // ==== Dynamically change cylinder colors
        this.my_Cylinder.setColors([0,0,0.5+(this.counter/100)%0.5], [0,1-(this.counter/100)%1.0,0]);        
    }
}

var app = new CylinderApp('CylinderCanvas');

app.run();
</script>

# Cone

The procedure to create the cone is in everything similar to the one to create a cylinder, but without the top base.

In [ ]:
%%html

<canvas id="ConeCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="ConeApp">

class cone extends CGRAobject
{
    // ===== Contructor
    constructor(glcontext, col1 = [0.0, 1.0, 0.0], col2 = [0.0, 0.0, 0.0], faces = 10)
    {
        // ===== Initialize the parent class
        super(glcontext); 
        
        // ===== Constants
        // Number of faces (3 to inf)
        this.sides = faces;
        this.numvertices = this.sides*3*2;
        this.length = 1; this.radius = 0.5;
        
        // ===== Calculate base
        let base = [];   
        for (let i = 0; i < this.sides; i++)
        { 
            let theta = i*2*Math.PI/this.sides;
            base.push(
                [this.radius*Math.cos(theta),
                 -1*(this.length)/2,
                this.radius*Math.sin(theta)]);
        }
        
        // ===== Calculate vertices
        var vertices = [];
        for (let i = 0; i < this.sides; i++)
        {   
            vertices.push(base[i]);
            vertices.push(base[(i+1)%this.sides]);
            vertices.push([0,-1*(this.length)/2,0]);
            
            vertices.push(base[i]);
            vertices.push(base[(i+1)%this.sides]);
            vertices.push([0,(this.length)/2,0]);
        }
        
        // ===== Assign colors
        this.Cone_colors = [col1, col1, col2].flat();
        let colors = [];
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.Cone_colors);
        }
        
        // ===== Populate buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices.flat()), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
    
    // Function to dynamically change colors
    setColors(newColor1, newColor2)
    {
        this.Cone_colors = [newColor1, newColor1, newColor2].flat();
        let colors = [];
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.Cone_colors);
        }
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
}


class ConeApp extends DEECapp
{
    counter=0;
    initialize()
    {
        // ===== Perform initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9, 0.9, 0.9, 1.0);
        
        // ===== Create cone
        this.my_cone = new cone(this.gl, [0.9, 0, 0] ,[1, 1, 0], 10);
        this.my_cone.setShader(this.shaderprog);

        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,10);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(2,1,0),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        this.shaderprog.startUsing();
        
        
        ////////////////// Transform/Animate Couple //////////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(0.0, 0.0, 1.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.my_cone.setModelTransformation(model);
        
        // ===== Draw cone
        this.my_cone.drawit(this.viewM,this.projectionM);
    }
}

var app = new ConeApp('ConeCanvas');

app.run();
</script>

# Disc

The procedure to create the cone is in everything similar to the one to create a cylinder or a cone, but just needs to create a base.

In [ ]:
%%html

<canvas id="DiscCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="DiscApp">

class disc extends CGRAobject
{
    // ===== Constructor
    constructor(glcontext, col1 = [0.0, 1.0, 0.0], col2 = [0.0, 0.0, 0.0], faces = 10)
    {
        // ===== Initialize the parent class
        super(glcontext); 
        
        // ===== Constants
        // Number of faces (3 to inf)
        this.sides = faces;
        this.numvertices = this.sides*3;
        this.radius = 0.5;
        
        // ===== Calculate vertices
        let base = [];   
        for (let i = 0; i < this.sides; i++)
        { 
            let theta = i*2*Math.PI/this.sides;
            base.push(
                [this.radius*Math.cos(theta),
                 0.0,
                this.radius*Math.sin(theta)]);
        }
        
        // ===== Assign vertices
        var vertices = [];
        for (let i = 0; i < this.sides; i++)
        {   
            vertices.push(base[i]);
            vertices.push(base[(i+1)%this.sides]);
            vertices.push([0,0,0]);
        }
        
        // ===== Assign colors
        this.Disc_colors = [col1, col1, col2].flat();
        let colors = [];
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.Disc_colors);
        }        
        
        // ===== Populate buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices.flat()), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
    
    // ===== Function to dinamically change colors of disc
    setColors(newColor1, newColor2)
    {
        this.Disc_colors = [newColor1, newColor1, newColor2].flat();
        let colors = [];
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.Disc_colors);
        }
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
}


class DiscApp extends DEECapp
{
    counter=0;
    initialize()
    {
        // ===== Perform other initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9,0.9,0.9,1.0);
        
        // ===== Create cone
        this.my_disc = new disc(this.gl, [0.9, 0, 0.5] ,[0, 1, 1], 7);
        this.my_disc.setShader(this.shaderprog);

        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,10);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(2,1,0),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        this.shaderprog.startUsing();
        
        ////////////////// Transform/Animate Couple //////////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Rotation
        var rotation = glm.radians(this.counter*5);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.my_disc.setModelTransformation(model);
        
        // ===== Draw Disc
        this.my_disc.drawit(this.viewM,this.projectionM);
    }
}

var app = new DiscApp('DiscCanvas');

app.run();
</script>

# Compound Shape

## Moon with Flag

In [ ]:
%%html

<canvas id="MoonCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="MoonApp">

//////////////////////////////// FLAG //////////////////////////////// 
class flag extends CGRAobject 
{
    countermoon = 0;
    
    // ===== Constructor
    constructor(glcontext)
    {
        super(glcontext);
        
        // ===== Instantiate components
        this.pole = new cylinder(this.gl,[1, 1, 0.58] ,[1, 0.8, 0.2], 20);
        this.center = new cylinder(this.gl,[1, 1, 0.0] ,[0.8, 0.9, 0.0], 20);
        this.flag = new cube(this.gl,[0.8, 0.2, 0.0] ,[0.2, 0.8, 0]);
    }
    
    // ===== Extend drawit
    drawit(viewM, projectM,parentMat4 = glm.mat4(1.0))
    {
        // ===== Falg global model matrix
        var globalMat = parentMat4['*'](this.modelMat);
 
        this.pole.setShader(this.shaderprog);
        this.flag.setShader(this.shaderprog);
        this.center.setShader(this.shaderprog);
        
        // ===== Pole Model
        var pole = glm.mat4(glm.mat3(1));
        /// ===== Scale
        var scale = glm.vec3(0.5, 3.1, 0.5);
        pole = glm.scale(pole, scale);
        // ===== Translate
        var translation = glm.vec3(0,0,0);
        pole = glm.translate(pole, translation);
        // ===== Final Transformation
        this.pole.setModelTransformation(pole);      
        
        // ===== Draw pole
        this.pole.drawit(viewM,projectM,globalMat);
        
        
        // ===== Flag Model
        var flag = glm.mat4(glm.mat3(1));
        /// ===== Scale
        var scale = glm.vec3(0.4, 1, 2);
        flag = glm.scale(flag, scale);
        // ===== Translate
        var translation = glm.vec3(0,1,-0.5);
        flag = glm.translate(flag, translation);
        // ===== Final Transformation
        this.flag.setModelTransformation(flag);  
        
        // ===== Draw flag
        this.flag.drawit(viewM,projectM,globalMat);
        
        
        // ===== Center Model
        var center = glm.mat4(glm.mat3(1));
        /// ===== Scale
        var scale = glm.vec3(0.45, 0.5, 0.5);
        center = glm.scale(center, scale);
        // ===== Translate
        var translation = glm.vec3(0,2,-2);
        center = glm.translate(center, translation);
        // ===== Rotation
        var rotation = glm.radians(90);
        var axis = glm.vec3(0.0,0,1.0);
        center = glm.rotate(center, rotation, axis);
        // ===== Final Transformation
        this.center.setModelTransformation(center);      
        
        // ===== Draw Center
        this.center.drawit(viewM,projectM,globalMat);
    }
}

//////////////////////////////// Moon //////////////////////////////// 
class moon extends CGRAobject 
{
    counter = 0;
    
    // ===== Constructor
    constructor(glcontext)
    {
        super(glcontext);
        
        // ===== Instantiate components
        this.moon = new sphere(this.gl,[0.58, 0.58, 0.58] ,[0.8, 0.8, 0.8], 20);
        this.flag = new flag(this.gl);
        
    }
    
    // ===== Extend drawit
    drawit(viewM, projectM, parentMat4 = glm.mat4(1.0))
    {
        // ===== Calculate global model matrix
        var globalMat = parentMat4['*'](this.modelMat);
        
        this.counter ++;
        
        this.moon.setShader(this.shaderprog);
        this.flag.setShader(this.shaderprog);
        
        // ===== Moon Model
        var moon = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(2.5, 2.5, 2.5);
        moon = glm.scale(moon, scale);
        // ===== Translate
        var translation = glm.vec3(0,0,0);
        moon = glm.translate(moon, translation);
        // ===== Final Transformation
        this.moon.setModelTransformation(moon);      
        
        // ===== Draw moon
        this.moon.drawit(viewM,projectM,globalMat);

        
        // ===== Flag Model
        var model = glm.mat4(glm.mat3(0.25));
        // ===== Scale
        var scale = glm.vec3(1.0);
        model = glm.scale(model, scale);
        // ===== Rotation
        var rotation = glm.radians(-45);
        var axis = glm.vec3(1.0, 0.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(0, 0.0, 1);
        model = glm.rotate(model, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,6,0);
        model = glm.translate(model, translation);
        // ===== Final Transformation
        this.flag.setModelTransformation(model);
        
        // ===== Draw flag
        this.flag.drawit(viewM,projectM,globalMat); 
    }
}


//////////////////////////////// SCENE //////////////////////////////// 
class MoonApp extends DEECapp
{
    counter=0;
    initialize()
    {
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9, 0.9, 0.9, 1.0);
        
        // ===== Create moon
        this.moon = new moon(this.gl);
        this.moon.setShader(this.shaderprog);
                
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,10);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(5,0,0),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;

        this.shaderprog.startUsing();
        
        // ===== Model
        var model = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(1.0, 1.0, 1.0);
        model = glm.scale(model, scale);
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.moon.setModelTransformation(model);
        
        // ===== Draw
        this.moon.drawit(this.viewM,this.projectionM);     
    }
}

var app = new MoonApp('MoonCanvas');

app.run();
</script>



## Couple

In [ ]:
%%html

<canvas id="CoupleCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="CoupleApp">

//////////////////////////////// COUPLE ////////////////////////////////
class couple extends CGRAobject
{
    // ===== Constructor
    constructor(glcontext)
    {
        super(glcontext);
        
        // ===== Instantiate components
        this.woman = new woman(this.gl);
        this.man = new man(this.gl);
    }
    
    // ===== Extend drawit
    drawit(viewM, projectionM, parentMat4 = glm.mat4(1.0))
    {    
        var couplemodeMat = parentMat4['*'](this.modelMat);
        
        this.man.setShader(this.shaderprog);
        this.woman.setShader(this.shaderprog);
        
        // ===== Draw components
        this.man.drawit(viewM,projectionM,couplemodeMat);
        this.woman.drawit(viewM,projectionM,couplemodeMat);
    }
}

//////////////////////////////// WOMAN ////////////////////////////////
class woman extends CGRAobject 
{
    // ===== Constructor
    constructor(glcontext)
    {
        super(glcontext);

        // ===== Instantiate components
        this.body = new cone(this.gl,[1,0.6,0.7],[1,0.8,0.9],10);
        this.head = new sphere(this.gl,[0.8,0.7,0.6],[0.5,0.3,0.1],20);
    }
    
    // ===== Extend drawit
    drawit(viewM, projectionM, parentMat4 = glm.mat4(1.0))
    {   
        // ===== Connect to parent model
        var globalMat = parentMat4['*'](this.modelMat);
 
        this.body.setShader(this.shaderprog);        
        this.head.setShader(this.shaderprog);
    
        // ===== Apply transformations to body
        var scale = glm.mat4(glm.mat3(1));
        var translate = glm.translate(glm.mat4(1.0),glm.vec3(0,0,0.5));
        var modelMat_body = translate["*"](scale);
        this.body.setModelTransformation(modelMat_body);
        this.body.drawit(viewM, projectionM, globalMat);
        
        // ===== Apply transformations to head
        var scale = glm.mat4(glm.mat3(0.5));
        var translate = glm.translate(glm.mat4(1.0),glm.vec3(0,0.7,0.5));
        var modelMat_head = translate['*'](scale);
        this.head.setModelTransformation(modelMat_head);
        this.head.drawit(viewM, projectionM, globalMat);
    }
}

//////////////////////////////// MAN //////////////////////////////// 
class man extends CGRAobject 
{
    // ===== Constructor
    constructor(glcontext)
    {
        // Initialize the parent class
        super(glcontext); 
        
        // ===== Instantiate components
        this.body = new cone(this.gl,[0,0,1],[0,0,0],10);
        this.head = new sphere(this.gl,[0.8,0.7,0.6],[0.5,0.3,0.1],20);
    }
    
    // ===== Extend drawit
    drawit(viewM, projectionM, parentMat4 = glm.mat4(1.0))
    {   
        // ===== Connect to parent model
        var globalMat = parentMat4['*'](this.modelMat);
        
        this.body.setShader(this.shaderprog);        
        this.head.setShader(this.shaderprog);
        
        // ===== Apply transformations to body
        var scale = glm.mat4(glm.mat3(1));
        translate = glm.translate(glm.mat4(1.0),glm.vec3(0,0,-0.5));
        var rotation = glm.toMat4(glm.angleAxis(glm.radians(180),glm.vec3(0.0,0.0,1.0)));
        var modelMat_body = translate["*"](rotation['*'](scale));
        this.body.setModelTransformation(modelMat_body);
        this.body.drawit(viewM, projectionM, globalMat);
        
        // ===== Apply transformations to head
        var scale = glm.mat4(glm.mat3(0.5));
        var translate = glm.translate(glm.mat4(1.0), glm.vec3(0,0.7,-0.5));
        var modelMat_head = translate['*'](scale);
        this.head.setModelTransformation(modelMat_head);
        this.head.drawit(viewM, projectionM, globalMat);
    }
}


//////////////////////////////// SCENE //////////////////////////////// 
class CoupleApp extends DEECapp
{
    counter=0;
    initialize()
    {
        // ===== Perform initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9, 0.9, 0.9, 1.0);
        
        // ===== Create couple
        this.couple = new couple(this.gl);
        this.couple.setShader(this.shaderprog);
        

        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,10);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(5,0,0),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        this.shaderprog.startUsing();
        
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(1.0, 1.0, 1.0);
        model = glm.scale(model, scale);
        // ===== Rotation
        var rotation = glm.radians(this.counter*3);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.couple.setModelTransformation(model);
        
        // ===== Draw couple
        this.couple.drawit(this.viewM,this.projectionM);   
    }
}

var app = new CoupleApp('CoupleCanvas');

app.run();
</script>



# Table

In [ ]:
%%html

<canvas id="TableCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="TableApp">

//////////////////////////////// TABLE //////////////////////////////// 
class table extends CGRAobject
{
    // ===== Constructor
    constructor(glcontext, col1 = [0.8,0.7,0.6], col2 =[0.6,0.3,0.1])
    {
        // ===== Initialize the parent class
        super(glcontext);
        
        // ===== Instantiate components
        this.top = new cube(this.gl, col1, col2);
        this.leg1 = new cylinder(this.gl, col1, col2, 10);
        this.leg2 = new cylinder(this.gl, col1, col2, 10);
        this.leg3 = new cylinder(this.gl, col1, col2, 10);
        this.leg4 = new cylinder(this.gl, col1, col2, 10)  ; 
    }
    
    // ===== Extend drawit
    drawit(viewM, projectM, parentMat4 = glm.mat4(1.0))
    {
        // ===== Connect to parent model
        var globalMat = parentMat4['*'](this.modelMat);
        
        this.top.setShader(this.shaderprog);        
        this.leg1.setShader(this.shaderprog);
        this.leg2.setShader(this.shaderprog);
        this.leg3.setShader(this.shaderprog);
        this.leg4.setShader(this.shaderprog);
        
        // ================ Top ==================
        var top = glm.mat4(glm.mat3(1));
        /// ===== Scale
        var scale = glm.vec3(2.5, 0.25, 2.5);
        top = glm.scale(top, scale);
        // ===== Translate
        var translation = glm.vec3(0,0,0);
        top = glm.translate(top, translation);
        // ===== Final Transformation
        this.top.setModelTransformation(top);      
        // ===== Draw
        this.top.drawit(viewM,projectM,globalMat);
        
        // ================ Leg1 ==================
        var leg1 = glm.mat4(glm.mat3(1));
        /// ===== Scale
        var scale = glm.vec3(0.25, 1.5, 0.25);
        leg1 = glm.scale(leg1, scale);
        // ===== Translate
        var translation = glm.vec3(4,-0.5,4);
        leg1 = glm.translate(leg1, translation);
        // ===== Final Transformation
        this.leg1.setModelTransformation(leg1);      
        // ===== Draw
        this.leg1.drawit(viewM,projectM,globalMat);
        
        // ================ Leg2 ==================
        var leg2 = glm.mat4(glm.mat3(1));
        /// ===== Scale
        var scale = glm.vec3(0.25, 1.5, 0.25);
        leg2 = glm.scale(leg2, scale);
        // ===== Translate
        var translation = glm.vec3(-4,-0.5,-4);
        leg2 = glm.translate(leg2, translation);
        // ===== Final Transformation
        this.leg2.setModelTransformation(leg2);      
        // ===== Draw
        this.leg2.drawit(viewM,projectM,globalMat);
        
        // ================ Leg3 ==================
        var leg3 = glm.mat4(glm.mat3(1));
        /// ===== Scale
        var scale = glm.vec3(0.25, 1.5, 0.25);
        leg3 = glm.scale(leg3, scale);
        // ===== Translate
        var translation = glm.vec3(-4,-0.5,4);
        leg3 = glm.translate(leg3, translation);
        // ===== Final Transformation
        this.leg3.setModelTransformation(leg3);      
        // ===== Draw
        this.leg3.drawit(viewM,projectM,globalMat);
        
        // ================ Leg4 ==================
        var leg4 = glm.mat4(glm.mat3(1));
        /// ===== Scale
        var scale = glm.vec3(0.25, 1.5, 0.25);
        leg4 = glm.scale(leg4, scale);
        // ===== Translate
        var translation = glm.vec3(4,-0.5,-4);
        leg4 = glm.translate(leg4, translation);
        // ===== Final Transformation
        this.leg4.setModelTransformation(leg4);      
        // ===== Draw
        this.leg4.drawit(viewM,projectM,globalMat);
    }
}

//////////////////////////////// SCENE //////////////////////////////// 
class TableApp extends DEECapp
{
    counter=0;
    initialize()
    {
        // ===== Perform initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9,0.9,0.9,1.0);
        
        // ===== Instantiate table
        this.table = new table(this.gl,[0.8,0.7,0.6],[0.6,0.3,0.1]);

        this.table.setShader(this.shaderprog);

 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,100);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(3,-1,5),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        this.shaderprog.startUsing();
        
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(1.0, 1.0, 1.0);
        model = glm.scale(model, scale);
        // ===== Translate
        //var translation = glm.vec3(Math.cos(this.counter/10),0,Math.sin(this.counter/10));
        //var translation = glm.vec3(0,0,0);
        //var translation = glm.vec3(0,2*Math.sin(this.counter/20),0);
        //model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter*3);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.table.setModelTransformation(model);  
        
        // ===== Draw
        this.table.drawit(this.viewM,this.projectionM);
    }
}

var app = new TableApp('TableCanvas');

app.run();
</script>

## Museum = Floor +  Walls +  Door + Windows

W - Front | A - Left | S - Back | D - Right | . - Reset

In [ ]:
%%html

<canvas id="MuseumCanvas" width="600" height="600" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="MuseumApp">

//////////////////////////////// WALL ////////////////////////////////
class wallRoom extends CGRAobject
{
    constructor(glcontext, color = [0.9, 0.9, 0.9])
    {
        super(glcontext);

        this.wallObj = new cube(this.gl, color, color);
    }
    
    drawit(viewM, projectionM, parentMat4 = glm.mat4(1.0))
    {   
        var globalMat = parentMat4['*'](this.modelMat);
 
        this.wallObj.setShader(this.shaderprog);        
    
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(1.0, 1.0, 0.05);
        model = glm.scale(model, scale);
        this.wallObj.setModelTransformation(model);
        this.wallObj.drawit(viewM, projectionM, globalMat);
    }
}

//////////////////////////////// DOOR //////////////////////////////// 
class doorRoom extends CGRAobject
{
    constructor(glcontext, color = [0.9, 0.9, 0.9])
    {
        super(glcontext);
        
        // ===== Instantiate components
        this.left = new cube(this.gl, color, color);
        this.right = new cube(this.gl, color, color);
        this.top = new cube(this.gl, color, color);
    }
    
    drawit(viewM, projectionM, parentMat4 = glm.mat4(1.0))
    {   
        // ===== Connect to parent
        var globalMat = parentMat4['*'](this.modelMat);
 
        this.left.setShader(this.shaderprog);
        this.right.setShader(this.shaderprog);
        this.top.setShader(this.shaderprog);
        
        ///////////// Left Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(-0.3,0,0);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(0.4, 1.0, 0.05);
        model = glm.scale(model, scale);
        this.left.setModelTransformation(model);
        this.left.drawit(viewM, projectionM, globalMat);
        
        ///////////// Right Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0.3,0,0);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(0.4, 1.0, 0.05);
        model = glm.scale(model, scale);
        this.right.setModelTransformation(model);
        this.right.drawit(viewM, projectionM, globalMat);
        
        ///////////// Top Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0,0.2,0);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(0.2, 0.6, 0.05);
        model = glm.scale(model, scale);
        this.top.setModelTransformation(model);
        this.top.drawit(viewM, projectionM, globalMat);
    }
}

//////////////////////////////// WINDOW //////////////////////////////// 
class windowRoom extends CGRAobject
{
    constructor(glcontext, color = [0.9, 0.9, 0.9])
    {
        super(glcontext);
        
        // ===== Instantiate components
        this.left = new cube(this.gl, color, color);
        this.right = new cube(this.gl, color, color);
        this.top = new cube(this.gl, color, color);
        this.bottom = new cube(this.gl, color, color);
        
    }
    
    drawit(viewM, projectionM, parentMat4 = glm.mat4(1.0))
    {   
        // ===== Connect to parent
        var globalMat = parentMat4['*'](this.modelMat);
 
        this.left.setShader(this.shaderprog);
        this.right.setShader(this.shaderprog);
        this.top.setShader(this.shaderprog);
        this.bottom.setShader(this.shaderprog);
        
        ///////////// Left Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(-0.35,0,0);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(0.3, 1.0, 0.05);
        model = glm.scale(model, scale);
        this.left.setModelTransformation(model);
        this.left.drawit(viewM, projectionM, globalMat);
        
        ///////////// Right Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0.35,0,0);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(0.3, 1.0, 0.05);
        model = glm.scale(model, scale);
        this.right.setModelTransformation(model);
        this.right.drawit(viewM, projectionM, globalMat);
        
        ///////////// Top Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0,0.25,0);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(0.4, 0.5, 0.05);
        model = glm.scale(model, scale);
        this.top.setModelTransformation(model);
        this.top.drawit(viewM, projectionM, globalMat);
        
        ///////////// Bottom Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0,-0.4,0);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(0.4, 0.2, 0.05);
        model = glm.scale(model, scale);  
        this.bottom.setModelTransformation(model);
        this.bottom.drawit(viewM, projectionM, globalMat);
    }
}

//////////////////////////////// MUSEUM //////////////////////////////// 
class museumRoom extends CGRAobject
{
    constructor(glcontext, colorful = 0)
    {
        // ===== Initialize super
        super(glcontext);
        
        // ===== Instantiate components
        if(colorful)
        {
            this.back = new wallRoom(this.gl,[1,0,0]);
            this.front = new doorRoom(this.gl,[0,1,0]);
            this.left = new windowRoom(this.gl,[0,0,1]);
            this.right = new windowRoom(this.gl,[1,1,0]);
            this.floor = new wallRoom(this.gl,[0.9,0.9,0.9]);
        }
        else
        {
            this.back = new wallRoom(this.gl,[0.9, 0.9, 0.7]);
            this.front = new doorRoom(this.gl,[0.9, 0.9, 0.7]);
            this.left = new windowRoom(this.gl,[0.9, 0.9, 0.7]);
            this.right = new windowRoom(this.gl,[0.9, 0.9, 0.7]);
            this.floor = new wallRoom(this.gl,[0.9, 0.7, 0.4]);
        }        
    }
    
    // ===== Extend drawit
    drawit(viewM, projectionM, parentMat4 = glm.mat4(1.0))
    {   
        // ===== Connect to parent
        var globalMat = parentMat4['*'](this.modelMat);
 
        this.back.setShader(this.shaderprog);
        this.front.setShader(this.shaderprog);
        this.left.setShader(this.shaderprog);
        this.right.setShader(this.shaderprog);
        this.floor.setShader(this.shaderprog);
        
        ///////////// Back Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0.0, 0.0, -0.525);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(1.0);
        model = glm.scale(model, scale);
        this.back.setModelTransformation(model);
        this.back.drawit(viewM, projectionM, globalMat);
        
        ///////////// Front Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0.0, 0.0, 0.525);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(1.0);
        model = glm.scale(model, scale);
        this.front.setModelTransformation(model);
        this.front.drawit(viewM, projectionM, globalMat);
        
        ///////////// Left Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(-0.525,0,0);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(90);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Scale
        var scale = glm.vec3([1.1, 1.0, 1.0]);
        model = glm.scale(model, scale);
        this.left.setModelTransformation(model);
        this.left.drawit(viewM, projectionM, globalMat);
        
        ///////////// Right Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0.525,0,0);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(90);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Scale
        var scale = glm.vec3([1.1, 1.0, 1.0]);
        model = glm.scale(model, scale);  
        this.right.setModelTransformation(model);
        this.right.drawit(viewM, projectionM, globalMat);
        
        ///////////// Floor Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0,-0.525,0);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(90);
        var axis = glm.vec3(1.0, 0.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Scale
        var scale = glm.vec3([1.1, 1.1, 1.0]);
        model = glm.scale(model, scale);  
        this.floor.setModelTransformation(model);
        this.floor.drawit(viewM, projectionM, globalMat);
    }
}

//////////////////////////////// SCENE //////////////////////////////// 
class MuseumApp extends DEECapp
{
    positionX = 0.0;
    positionZ = 5.0;
    camera_angle = glm.radians(270);
    counter = 0;
    initialize()
    {
        // ===== Perform initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.1,0.1,0.1,1.0);
        
        // ===== Create museum object
        var colorful = 0;
        this.building = new museumRoom(this.gl, colorful);
        this.building.setShader(this.shaderprog);

        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(100),1.0,0.001,10000);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(0,0,3),glm.vec3(0,0,0),glm.vec3(0,1,0));           
        
        window.addEventListener("keypress",(evt)=>this.keyprocess(evt),false);
    }
    
    // ===== Process key presses
    keyprocess(evt)
    {
        switch (evt.keyCode)
        {
             case 27:
                alert("You pressed the \"Escape\" key."); 
                break;
             case 100: // D - Right
                 this.camera_angle = this.camera_angle + 0.1;
                 break;
             case 97:  // A - Left
                 this.camera_angle = this.camera_angle - 0.1;
                 break;
             case 115: // S - Back
                 this.positionZ-= 20*Math.sin(this.camera_angle);
                 this.positionX -= 20*Math.cos(this.camera_angle);
                 break;
             case 119: // W - Front
                 this.positionZ+= 20*Math.sin(this.camera_angle);;
                 this.positionX += 20*Math.cos(this.camera_angle);
                 break;
             case 46:  // . - Reset camera position
                 this.positionZ = 5;
                 this.positionX = 0;
                 this.camera_angle = glm.radians(270);
        }
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        this.shaderprog.startUsing();
        
        // ===== Allow movement of "player"
        this.viewM = glm.lookAt(glm.vec3(this.positionX,0.5,this.positionZ),
                                glm.vec3(this.positionX + Math.cos(this.camera_angle),
                                         0.5,
                                         this.positionZ + Math.sin(this.camera_angle)),
                                glm.vec3(0,1,0));        
        
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0,100,0);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter*3);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        //model = glm.rotate(model, rotation, axis);
        // ===== Scale
        var scale = glm.vec3(500.0);
        model = glm.scale(model, scale);
        // ===== Final Transformation
        this.building.setModelTransformation(model);
        
        // ===== Draw museum
        this.building.drawit(this.viewM,this.projectionM); 
    }
}

var app = new MuseumApp('MuseumCanvas');

app.run();
</script>



## Solar System

1 - Stop | 2 - Slow | 3 - Normal | 4 - Fast

R - Spheres | T - Cubes | Y - Cylinders | U - Cones

In [ ]:
%%html

<canvas id="myCCanvas_solarSystem" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myCapp_solarSystem">

class solarSystem extends CGRAobject
{
    form = 1;
    constructor(glcontext, form = 1)
    {    
        super(glcontext)
        
        this.form = form;
        
        switch(form)
        {
            case 1: 
                this.Sun = new sphere(this.gl,[1, 0.7, 0] ,[1, 1, 0], 20);
                this.Mercury = new sphere(this.gl,[0.58, 0.58, 0.58] ,[0.8, 0.8, 0.8], 20);
                this.Venus = new sphere(this.gl,[0.58, 0.29, 0.1] ,[1, 0.8, 0.67], 20);
                this.Earth = new sphere(this.gl,[0, 0.8, 0.3], [0, 0, 1], 20);
                this.Mars = new sphere(this.gl,[0.9, 0, 0] ,[1, 0.5, 0], 20);
                this.Jupiter = new sphere(this.gl,[0.9, 0.4, 0] ,[0.8, 0.8, 0.8], 20);
                this.Saturn = new sphere(this.gl,[0.8, 0.4, 0] ,[0.8, 0.8, 0.8], 20);
                this.Uranus = new sphere(this.gl,[0.4, 0.6, 0.9] ,[0, 0.7, 1], 20);
                this.Neptune = new sphere(this.gl,[0, 0, 1] ,[0, 0.5, 1], 20);
                break;
            case 2:
                this.Sun = new cube(this.gl,[1, 0.7, 0] ,[1, 1, 0]);
                this.Mercury = new cube(this.gl,[0.58, 0.58, 0.58] ,[0.8, 0.8, 0.8]);
                this.Venus = new cube(this.gl,[0.58, 0.29, 0.1] ,[1, 0.8, 0.67]);
                this.Earth = new cube(this.gl,[0, 0.8, 0.3], [0, 0, 1]);
                this.Mars = new cube(this.gl,[0.9, 0, 0] ,[1, 0.5, 0]);
                this.Jupiter = new cube(this.gl,[0.9, 0.4, 0] ,[0.8, 0.8, 0.8]);
                this.Saturn = new cube(this.gl,[0.8, 0.4, 0] ,[0.8, 0.8, 0.8]);
                this.Uranus = new cube(this.gl,[0.4, 0.6, 0.9] ,[0, 0.7, 1]);
                this.Neptune = new cube(this.gl,[0, 0, 1] ,[0, 0.5, 1]);
                break;
            case 3:
                this.Sun = new cylinder(this.gl,[1, 0.7, 0] ,[1, 1, 0], 10);
                this.Mercury = new cylinder(this.gl,[0.58, 0.58, 0.58] ,[0.8, 0.8, 0.8], 10);
                this.Venus = new cylinder(this.gl,[0.58, 0.29, 0.1] ,[1, 0.8, 0.67], 10);
                this.Earth = new cylinder(this.gl,[0, 0.8, 0.3], [0, 0, 1], 10);
                this.Mars = new cylinder(this.gl,[0.9, 0, 0] ,[1, 0.5, 0], 10);
                this.Jupiter = new cylinder(this.gl,[0.9, 0.4, 0] ,[0.8, 0.8, 0.8], 10);
                this.Saturn = new cylinder(this.gl,[0.8, 0.4, 0] ,[0.8, 0.8, 0.8], 10);
                this.Uranus = new cylinder(this.gl,[0.4, 0.6, 0.9] ,[0, 0.7, 1], 10);
                this.Neptune = new cylinder(this.gl,[0, 0, 1] ,[0, 0.5, 1], 10);
                break;
            case 4:
                this.Sun = new cone(this.gl,[1, 0.7, 0] ,[1, 1, 0], 10);
                this.Mercury = new cone(this.gl,[0.58, 0.58, 0.58] ,[0.8, 0.8, 0.8], 10);
                this.Venus = new cone(this.gl,[0.58, 0.29, 0.1] ,[1, 0.8, 0.67], 10);
                this.Earth = new cone(this.gl,[0, 0.8, 0.3], [0, 0, 1], 10);
                this.Mars = new cone(this.gl,[0.9, 0, 0] ,[1, 0.5, 0], 10);
                this.Jupiter = new cone(this.gl,[0.9, 0.4, 0] ,[0.8, 0.8, 0.8], 10);
                this.Saturn = new cone(this.gl,[0.8, 0.4, 0] ,[0.8, 0.8, 0.8], 10);
                this.Uranus = new cone(this.gl,[0.4, 0.6, 0.9] ,[0, 0.7, 1], 10);
                this.Neptune = new cone(this.gl,[0, 0, 1] ,[0, 0.5, 1], 10);
                break;
            }
        
        this.SaturnDisc = new disc(this.gl, [0.9, 0.7, 0] ,[0.7, 0.7, 0.6], 20);
            
        }
        
    
    drawit(viewM, projectM, counter, parentMat4 = glm.mat4(1.0))
    {
        var globalMat = parentMat4['*'](this.modelMat);
        
        this.Sun.setShader(this.shaderprog);
        this.Mercury.setShader(this.shaderprog);
        this.Venus.setShader(this.shaderprog);
        this.Earth.setShader(this.shaderprog);
        this.Mars.setShader(this.shaderprog);
        this.Jupiter.setShader(this.shaderprog);
        this.Saturn.setShader(this.shaderprog);
        this.Uranus.setShader(this.shaderprog);
        this.Neptune.setShader(this.shaderprog);
        
        
        ////////////////////////SUN//////////////////////////////
         var Sun = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(2.5,2.5, 2.5);
        Sun = glm.scale(Sun, scale);
        // ===== Rotation
        var rotation = glm.radians(-counter);
        var axis = glm.vec3(0.0,1.0,0.0);
        Sun = glm.rotate(Sun, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,0);
        Sun = glm.translate(Sun, translation);
        // ===== Final Transformation
        this.Sun.setModelTransformation(Sun);      
        // ===== Draw
        this.Sun.drawit(viewM,projectM, globalMat);
        
        
        //////////////////////MERCURY///////////////////////////////////
        var Mercury = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(0.024, 0.024, 0.024);
        Mercury = glm.scale(Mercury, scale);
        // ===== Rotation
        var outer_rotation = counter*365/85;
        var rotation = glm.radians(outer_rotation);
        var axis = glm.vec3(0.0,1.0,0.0);
        Mercury = glm.rotate(Mercury, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,-70);
        Mercury = glm.translate(Mercury, translation);
        // ===== Rotation
        Mercury = glm.rotate(Mercury, rotation, axis);
        // ===== Final Transformation
        this.Mercury.setModelTransformation(Mercury);      
        // ===== Draw
        this.Mercury.drawit(viewM,projectM, globalMat);
        
        
        /////////////////////VENUS/////////////////////////////////////
        var Venus = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(0.121/2, 0.121/2, 0.121/2);
        Venus = glm.scale(Venus, scale);
        // ===== Rotation
        var outer_rotation = counter*365/224;
        var rotation = glm.radians(outer_rotation);
        var axis = glm.vec3(0.0,1.0,0.0);
        Venus = glm.rotate(Venus, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,-40);
        Venus = glm.translate(Venus, translation);
        // ===== Rotation
        var inner_rotation = counter/365;
        var rotation = glm.radians(inner_rotation);
        Venus = glm.rotate(Venus, rotation, axis);
        // ===== Final Transformation
        this.Venus.setModelTransformation(Venus);      
        // ===== Draw
        this.Venus.drawit(viewM,projectM, globalMat);
        
        
        ///////////////////////EARTH/////////////////////////////////////
        var Earth = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(0.127/2, 0.127/2, 0.127/2);
        Earth = glm.scale(Earth, scale);
        // ===== Rotation
        var outer_rotation = counter*365/365;
        var rotation = glm.radians(outer_rotation);
        var axis = glm.vec3(0.0,1.0,0.0);
        Earth = glm.rotate(Earth, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,-50);
        Earth = glm.translate(Earth, translation);
        // ===== Rotation
        var inner_rotation = counter/365;
        var rotation = glm.radians(inner_rotation);
        Earth = glm.rotate(Earth, rotation, axis);
        // ===== Final Transformation
        this.Earth.setModelTransformation(Earth);      
        // ===== Draw
        this.Earth.drawit(viewM,projectM, globalMat);
        if(this.form == 1)
            this.Earth.update_pos(globalMat);
        
        
        ///////////////////////MARS/////////////////////////////////
        var Mars = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(0.067/2, 0.067/2, 0.067/2);
        Mars = glm.scale(Mars, scale);
        // ===== Rotation
        var outer_rotation = counter*365/(365*1.88);
        var rotation = glm.radians(outer_rotation);
        var axis = glm.vec3(0.0,1.0,0.0);
        Mars = glm.rotate(Mars, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,-120);
        Mars = glm.translate(Mars, translation);
        // ===== Rotation
        Mars = glm.rotate(Mars, rotation, axis);
        // ===== Final Transformation
        this.Mars.setModelTransformation(Mars);      
        // ===== Draw
        this.Mars.drawit(viewM,projectM, globalMat);
        
        
        //////////////////////JUPITER///////////////////////////////
        var Jupiter = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(1.42/2, 1.42/2, 1.42/2);
        Jupiter = glm.scale(Jupiter, scale);
        // ===== Rotation
        var outer_rotation = counter/11.86;
        var rotation = glm.radians(outer_rotation);
        var axis = glm.vec3(0.0,1.0,0.0);
        Jupiter = glm.rotate(Jupiter, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,-8);
        Jupiter = glm.translate(Jupiter, translation);
        // ===== Rotation
        Jupiter = glm.rotate(Jupiter, rotation, axis);
        // ===== Final Transformation
        this.Jupiter.setModelTransformation(Jupiter);      
        // ===== Draw
        this.Jupiter.drawit(viewM,projectM, globalMat);
        
        
        //////////////////////SATURN///////////////////////////////
        this.SaturnDisc.setShader(this.shaderprog);
        var Saturn = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(1.2/2, 1.2/2, 1.2/2);
        Saturn = glm.scale(Saturn, scale);
        // ===== Rotation
        var outer_rotation = counter/29.46;
        var rotation = glm.radians(outer_rotation);
        var axis = glm.vec3(0.0,1.0,0.0);
        Saturn = glm.rotate(Saturn, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,-12);
        Saturn = glm.translate(Saturn, translation);
        // ===== Rotation
        Saturn = glm.rotate(Saturn, rotation, axis);
        // ===== Final Transformation
        this.Saturn.setModelTransformation(Saturn);
        var scale = glm.vec3(1.5, 1.5, 1.5);
        Saturn = glm.scale(Saturn, scale);
        this.SaturnDisc.setModelTransformation(Saturn);
        
        // ===== Draw
        this.Saturn.drawit(viewM,projectM, globalMat);
        this.SaturnDisc.drawit(viewM,projectM, globalMat);
        
        
        
        //////////////////////URANUS/////////////////////////////
        var Uranus = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(0.51/2, 0.51/2, 0.51/2);
        Uranus = glm.scale(Uranus, scale);
        // ===== Rotation
        var outer_rotation = counter/84.01;
        var rotation = glm.radians(outer_rotation);
        var axis = glm.vec3(0.0,1.0,0.0);
        Uranus = glm.rotate(Uranus, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,-36);
        Uranus = glm.translate(Uranus, translation);
        // ===== Rotation
        Uranus = glm.rotate(Uranus, rotation, axis);
        // ===== Final Transformation
        this.Uranus.setModelTransformation(Uranus);      
        // ===== Draw
        this.Uranus.drawit(viewM,projectM, globalMat);
        
        
        /////////////////////NEPTUNE////////////////////////////
        var Neptune = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(0.495/2, 0.495/2, 0.495/2);
        Neptune = glm.scale(Neptune, scale);
        // ===== Rotation
        var outer_rotation = counter/164.79;
        var rotation = glm.radians(outer_rotation);
        var axis = glm.vec3(0.0,1.0,0.0);
        Neptune = glm.rotate(Neptune, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,-44);
        Neptune = glm.translate(Neptune, translation);
        // ===== Rotation
        Neptune = glm.rotate(Neptune, rotation, axis);
        // ===== Final Transformation
        this.Neptune.setModelTransformation(Neptune);      
        // ===== Draw
        this.Neptune.drawit(viewM,projectM, globalMat);  
    }
}


class myCapp_solarSystem extends DEECapp{
    counter=0;
    speedup = 1;
    initialize(){
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.1,0.1,0.1,1.0);
        
        this.solarSystem = new solarSystem(this.gl, 1);
        this.solarSystem.setShader(this.shaderprog);

        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionMa= glm.perspective(glm.radians(60),1.0,0.1,1000);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(0, 0, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, -1): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewMa = glm.lookAt(glm.vec3(0,0,5),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    
        window.addEventListener("keypress",(evt)=>this.keyprocess(evt),false);
    }
    
    keyprocess(evt)
    {
        //console.log('Key pressed. Keycode:'+evt.keyCode + ' Char: \'' + String.fromCharCode(evt.charCode)+'\'');
         switch (evt.keyCode) {
             case 27:
                alert("You pressed the \"Escape\" key."); 
                break;
             case 49: // 1 Stop
                 this.speedup = 0;     
                 break;
             case 50: // 2 Slow
                 this.speedup = 0.2;
                 break;
             case 51: // 3 Middle
                 this.speedup = 1;
                 break;
             case 52: // 4 Fast
                 this.speedup = 10;
                 break;
             case 114: // r
                 this.solarSystem = new solarSystem(this.gl, 1);
                 this.solarSystem.setShader(this.shaderprog);
                 break;
             case 116: // t
                 this.solarSystem = new solarSystem(this.gl, 2);
                 this.solarSystem.setShader(this.shaderprog);
                 break;
             case 121: // y
                 this.solarSystem = new solarSystem(this.gl, 3);
                 this.solarSystem.setShader(this.shaderprog);
                 break;
             case 117: // u
                 this.solarSystem = new solarSystem(this.gl, 4);
                 this.solarSystem.setShader(this.shaderprog);
                 break;
        }
    }
    
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter = this.counter + this.speedup;
        
        this.shaderprog.startUsing();
        
        // Draw
        this.solarSystem.drawit(this.viewMa,this.projectionMa, this.counter);        
    }
}

var app2 = new myCapp_solarSystem('myCCanvas_solarSystem');

app2.run();
</script>

# Solar System Scene

W - Front | A - Left | S - Back | D - Right | . - Reset

1 - Stop | 2 - Slow | 3 - Normal | 4 - Fast

R - Spheres | T - Cubes | Y - Cylinders | U - Cones

In [ ]:
%%html

<canvas id="SolarSystemScene" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="SolarSystemScene">


class SolarSystemScene extends DEECapp{
    counter=0;
    speedup = 1;
    positionX = -5.0;
    positionZ = 0.0;
    camera_angle = 0;
    initialize()
    {
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.1,0.1,0.1,1.0);
        
        // Lets create solar system
        this.solarSystem = new solarSystem(this.gl, 1);
        this.solarSystem.setShader(this.shaderprog);
        
        
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(90),1.0,0.1,1000);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(0, 0, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, -1): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
                  
    
        window.addEventListener("keypress",(evt)=>this.keyprocess(evt),false);
    }
    
     keyprocess(evt)
    {
        //console.log('Key pressed. Keycode:'+evt.keyCode + ' Char: \'' + String.fromCharCode(evt.charCode)+'\'');
         switch (evt.keyCode) {
             case 27:
                alert("You pressed the \"Escape\" key."); 
                break;
             case 49: // 1 Stop
                 this.speedup = 0;
                 break;
             case 50: // 2 Slow
                 this.speedup = 0.2;
                 break;
             case 51: // 3 Middle
                 this.speedup = 1;
                 break;
             case 52: // 4 Fast
                 this.speedup = 10;
                 break;
             case 114: // r
                 this.solarSystem = new solarSystem(this.gl, 1);
                 this.solarSystem.setShader(this.shaderprog);
                 break;
             case 116: // t
                 this.solarSystem = new solarSystem(this.gl, 2);
                 this.solarSystem.setShader(this.shaderprog);
                 break;
             case 121: // y
                 this.solarSystem = new solarSystem(this.gl, 3);
                 this.solarSystem.setShader(this.shaderprog);
                 break;
             case 117: // u
                 this.solarSystem = new solarSystem(this.gl, 4);
                 this.solarSystem.setShader(this.shaderprog);
                 break;
             case 100: //d
                 this.camera_angle = this.camera_angle + 0.1;
                 //console.log("camera angle: " + this.camera_angle);
                 break;
             case 97: //a
                 this.camera_angle = this.camera_angle - 0.1;
                 //console.log("camera angle: " + this.camera_angle);
                 break;
             case 115: // s
                 this.positionZ-= 0.2*Math.sin(this.camera_angle);
                 this.positionX -= 0.2*Math.cos(this.camera_angle);
                 //console.log("Position X: " + this.positionX);
                 //console.log("Position Z: " + this.positionZ);
                 break;
             case 119: //w
                 this.positionZ+= 0.2*Math.sin(this.camera_angle);;
                 this.positionX += 0.2*Math.cos(this.camera_angle);
                 //console.log("Position X: " + this.positionX);
                 //console.log("Position Z: " + this.positionZ);
                 break;
             case 46: //. reset camera position
                 this.positionZ = 0;
                 this.positionX = -5;
                 this.camera_angle = 0;
             
             
        }
    }
    
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter = this.counter + this.speedup;
        
        this.shaderprog.startUsing();
        
        this.viewM = glm.lookAt(glm.vec3(this.positionX,0.5,this.positionZ),glm.vec3(this.positionX + Math.cos(this.camera_angle),0.5,this.positionZ + Math.sin(this.camera_angle)),glm.vec3(0,1,0)); 
        
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(1.0, 1.0, 1.0);
        model = glm.scale(model, scale);
        // ===== Translate
        //var translation = glm.vec3(Math.cos(this.counter/10),0,Math.sin(this.counter/10));
        //var translation = glm.vec3(0,0,0);
        //var translation = glm.vec3(0,1*Math.sin(this.counter/20),0);
        //model = glm.translate(model, translation);
        // ===== Rotation
        //var rotation = glm.radians(this.counter*3);
        //var axis = glm.vec3(0.0, 1.0, 0.0);
        //model = glm.rotate(model, rotation, axis);
        
        // ===== Final Transformation
        this.solarSystem.setModelTransformation(model);  
        
        // ===== Draw
        this.solarSystem.drawit(this.viewM,this.projectionM, this.counter);
        
    }
}

var app = new SolarSystemScene('SolarSystemScene');

app.run();
</script>

# Final Scene

W - Front | A - Left | S - Back | D - Right | . - Reset

1 - Stop | 2 - Slow | 3 - Normal | 4 - Fast (Only applies to Solar System)

R - Spheres | T - Cubes | Y - Cylinders | U - Cones

Contains:
- 1 solar system
- 1 table 
- 1 moon with flag
- 1 guard (man)
- 1 couple
- 1 museum room

In [ ]:
%%html

<canvas id="myScene" width="600" height="600" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myScene">


class myScene extends DEECapp{
    counter = 0;
    orbits = 0;
    speedup = 1;
    positionX = -15;
    positionZ = 15;
    camera_angle = glm.radians(-45);
    positionY = 0.033;
    view_flag = 1;
    initialize()
    {
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        // Initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.1,0.5,0.9,1.0);
        
        // Create Objects
        this.solarSystem = new solarSystem(this.gl);
        this.solarSystem.setShader(this.shaderprog);
        
        this.museum = new museumRoom(this.gl);
        this.museum.setShader(this.shaderprog);
        
        this.expositor = new table(this.gl,[0.3,0.1,0.05]);
        this.expositor.setShader(this.shaderprog);
        
        this.moon = new moon(this.gl);
        this.moon.setShader(this.shaderprog);
        
        this.guard = new man(this.gl);
        this.guard.setShader(this.shaderprog);
        
        this.couple = new couple(this.gl);
        this.couple.setShader(this.shaderprog);
        
        
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(100),1.0,0.001,1000);         
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(0, 0, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, -1): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////     
    
        window.addEventListener("keypress",(evt)=>this.keyprocess(evt),false);
    }
    
     keyprocess(evt)
    {
        //console.log('Key pressed. Keycode:'+evt.keyCode + ' Char: \'' + String.fromCharCode(evt.charCode)+'\'');
         switch (evt.keyCode) {
             case 27:
                alert("You pressed the \"Escape\" key."); 
                break;
             case 49: // 1 Stop
                 this.speedup = 0;
                 break;
             case 50: // 2 Slow
                 this.speedup = 0.2;
                 break;
             case 51: // 3 Middle
                 this.speedup = 1;
                 break;
             case 52: // 4 Fast
                 this.speedup = 10;
                 break;
            case 114: // r
                 this.solarSystem = new solarSystem(this.gl, 1);
                 this.solarSystem.setShader(this.shaderprog);
                 break;
             case 116: // t
                 this.solarSystem = new solarSystem(this.gl, 2);
                 this.solarSystem.setShader(this.shaderprog);
                 break;
             case 121: // y
                 this.solarSystem = new solarSystem(this.gl, 3);
                 this.solarSystem.setShader(this.shaderprog);
                 break;
             case 117: // u
                 this.solarSystem = new solarSystem(this.gl, 4);
                 this.solarSystem.setShader(this.shaderprog);
                 break;
             case 100: //d
                 this.camera_angle = this.camera_angle + 0.1;
                 break;
             case 97: //a
                 this.camera_angle = this.camera_angle - 0.1;
                 break;
             case 115: // s
                 this.positionZ-= 2*Math.sin(this.camera_angle);
                 this.positionX -= 2*Math.cos(this.camera_angle);
                 break;
             case 119: //w
                 this.positionZ+= 2*Math.sin(this.camera_angle);;
                 this.positionX += 2*Math.cos(this.camera_angle);
                 break;
             
             case 118: //v
                 if(this.view_flag == 1)
                     this.view_flag = 0;
                 else if(this.view_flag == 0)
                     this.view_flag = 1;
                 
                 break;

             case 46: //. reset camera position
                 if (this.view_flag == 1)
                 {
                     this.positionZ = 15;
                     this.positionX = -15;
                     this.camera_angle = glm.radians(-45);
                 }
        }
    }
    
    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        // ===== Update Speed
        this.orbits = this.orbits + this.speedup;
        this.counter++;
        
        this.shaderprog.startUsing();
        
        ////////////////// Transform/Animate Solar System //////////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        //var translation = glm.vec3(0, 0, 0);
        //model = glm.translate(model, translation);
        // ===== Scale
        //var scale = glm.vec3(1);
        //model = glm.scale(model, scale);
        // ===== Final Transformation
        this.solarSystem.setModelTransformation(model);  
        
        ////////////////// Transform/Animate Museum //////////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0,5,0);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3([40.0, 20.0, 40.0]);
        model = glm.scale(model, scale);
        // ===== Final Transformation
        this.museum.setModelTransformation(model);
        
        ////////////////// Transform/Animate Expositor //////////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0,-2,0);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter*3);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        //model = glm.rotate(model, rotation, axis);
        // ===== Scale
        var scale = glm.vec3([1, 2, 1]);
        model = glm.scale(model, scale);
        // ===== Final Transformation
        this.expositor.setModelTransformation(model);
        
        ////////////////// Transform/Animate Moon //////////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(-15, 2*Math.sin(this.counter/10), -15);
        model = glm.translate(model, translation);
        //var rotation = glm.radians(this.counter*5);
        //var axis = glm.vec3(0.0, 1.0, 0.0);
        //model = glm.rotate(model, rotation, axis);
        // ===== Scale
        //var scale = glm.vec3(1);
        //model = glm.scale(model, scale);
        // ===== Final Transformation
        this.moon.setModelTransformation(model);
        
        ////////////////// Transform/Animate Guard //////////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(15 - 2*(Math.sin(this.counter/15)+1), -2, 15 - 2*(Math.sin(this.counter/10)+1));
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(4);
        model = glm.scale(model, scale);
        // ===== Final Transformation
        this.guard.setModelTransformation(model);
        
        ////////////////// Transform/Animate Couple //////////////////
        var model = glm.mat4(glm.mat3(1.0));
        var translation = glm.vec3(3*Math.sin(this.counter/20), 0, 3*Math.sin(this.counter/20));
        model = glm.translate(model, translation);
        // ===== Translate
        var translation = glm.vec3(-9, -2, 9);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter*3);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Scale
        var scale = glm.vec3(3.5);
        model = glm.scale(model, scale);
        // ===== Final Transformation
        this.couple.setModelTransformation(model);
        
        
        if(this.view_flag == 1)
        {
            ////////////////// Visiter View //////////////////
            this.viewM = glm.lookAt(glm.vec3(this.positionX, 0.5, this.positionZ),
                                    glm.vec3(this.positionX + Math.cos(this.camera_angle), 
                                             0.5, 
                                             this.positionZ + Math.sin(this.camera_angle)),
                                    glm.vec3(0,1,0)); 
        }
        else
        {   
            ////////////////// Planet View //////////////////
            var Earth = glm.mat4(glm.mat3(1));
            // ===== Scale
            var scale = glm.vec3(0.127/2, 0.127/2, 0.127/2);
            Earth = glm.scale(Earth, scale);
            // ===== Rotation
            var outer_rotation = this.orbits;
            var rotation = glm.radians(outer_rotation);
            var axis = glm.vec3(0.0,1.0,0.0);
            Earth = glm.rotate(Earth, rotation, axis);
            // ===== Translate
            var translation = glm.vec3(0,0,-50);
            Earth = glm.translate(Earth, translation);
            // ===== Rotation
            var inner_rotation = this.orbits;
            var rotation = glm.radians(-inner_rotation);
            Earth = glm.rotate(Earth, rotation, axis);
            var axis = glm.vec3(0.0,1.0,0.0);
            Earth = glm.rotate(Earth, rotation, axis);   
            
            //var center = this.solarSystem.Earth.center;
            //console.log("Center: " + center);
            //var north = this.solarSystem.Earth.north_pole;
            //console.log("North: " + north);
            
            //this.viewM = glm.lookAt(glm.vec3(center[0], center[1], center[2]),
            //                        glm.vec3(0,this.positionY,0),
            //                        glm.vec3(0,1,0));
            
            this.viewM = glm.lookAt(glm.vec3(Earth[3][0], this.positionY,Earth[3][2]),
                                    glm.vec3(0,this.positionY,0),
                                    glm.vec3(0,1,0));
        }
        
        
        // ===== Draw
        this.solarSystem.drawit(this.viewM,this.projectionM, this.orbits);
        this.museum.drawit(this.viewM,this.projectionM);
        this.expositor.drawit(this.viewM,this.projectionM);
        this.moon.drawit(this.viewM,this.projectionM);
        this.guard.drawit(this.viewM,this.projectionM);
        this.couple.drawit(this.viewM,this.projectionM);
        
    }
}

var app2 = new myScene('myScene');

app2.run();
</script>